In [147]:
import tensorflow as tf
import numpy as np
import sys,time,os
from bLSTM import bLSTM


In [172]:
# Understand how to initialize 2 Tensorflow models,
# train them in one session, and save them independently


class model(object):
    
    def __init__(self,size):
        
        self.inp = tf.placeholder(tf.float32,(size,2),'inp')
        self.mat = tf.placeholder(tf.float32,(2,5),'mat')
        
        
        
    def foo(self):
        self.out = tf.matmul(self.inp,self.mat)
        
    
    

In [184]:
small = model(3)
small.foo()
large = model(10)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver(max_to_keep=4)

    
    b = sess.run(small.out, feed_dict={small.inp:np.random.random((3,2)),
                                      small.mat:np.random.random((2,5))})
    print(b)
    
    saver.save(sess, save_path + '/Model', global_step=epoch, write_meta_graph=False)

ValueError: No variables to save

In [198]:
def np_dict_to_dict(np_dict):
    """
    Converts a dictionary saved via np.save (as structured np array) into an object of type dict

    Parameters:
    --------------
    NP_DICT        : {np.array} structured np.array with dict keys and items

    Returns:
    --------------
    DICT            : {dict} converted NP_DICT

    """

    return {key:np_dict.item().get(key) for key in np_dict.item()}



def extract_celex(path):
    """
    Reads in data from the CELEX corpus
    
    Parameters:
    -----------
    PATH        {str} the path to the desired celex file, i.e. gpl.cd 
                    (contains orthography and phonology)

    Returns:
    -----------
    2 Tuples, each with 2 variables. 
        First tuple:
    W           {np.array} of words (length 51728) for gpl.cd
    P           {np.array} of phoneme sequences (length 51728) for gpl.cd
        Second tuple:
    WORD_DICT   {dict} allowing to map the numerical array W back to strings
    PHON_DICT   {dict} doing the same for the phonetical arrays P

    
    Call via:
    path = "/Users/jannisborn/Desktop/LDS_Data/celex2/german/gpl/gpl.cd"
    ((w,p) , (word_dict, phon_dict)) = extract_celex(path)
    
    """
    
    
    with open(path, 'r') as file:

        raw_data = file.read().splitlines()
        words = []
        phons = []
        
        for ind,raw_line in enumerate(raw_data):
            
            line = raw_line.split("\\")

            if line[-2]: # Use only words that HAVE a SAMPA transcript (reduces from 51k to 37345)

            # exclude foreign words that have the 'æ' tone (SAMPA '{' ) like in PoINte   - 18 words
            # exclude foreign words that have the 'ɑ' tone (SAMPA 'A' ) like in NuANce   - 28 words
            # exclude foreign words that have a nasal vowel (SAMPA '~' ) like in Jargon  - 22 words
                if not 'A' in line[-2] and not '{' in line[-2] and not '~' in line[-2]: 

                    if not ('tS' in line[-2] and not 'tsch' in line[1]): # exclude 9 foreign words like 'Image', 'Match', 'Punch', 'Sketch'

                        if len(line[1]) < 15: # exclude extra long words (reduces to 34376)

                            words.append(line[1].lower()) # All words are lowercase only
                            phons.append(line[-2]) # Using SAMPA notation

    return words,phons



def str_to_num_dataset(X,Y):
    """
    This method receives 2 lists of strings (input X and output Y) and converts it to padded, numerical arrays.
    It returns the numerical dataset as well as the dictionaries to retrieve the strings.
    """

    # 1. Define dictionaries 
    # Dictionary assignining a unique integer to each input character
    try:
        u_characters = set(' '.join(X)) 
    except TypeError:
        # Exception for TIMIT dataset (one phoneme is repr. by seq. of chars)
        print("TypeError occurred.")
        u_characters = set([quant for seq in X for quant in seq])

    char2numX = dict(zip(u_characters, range(1,len(u_characters)+1)))

    # Dictionary assignining a unique integer to each phoneme
    try:
        v_characters = set(' '.join(Y)) 
    except TypeError:
        print("TypeError occurred.")
        v_characters = set([quant for seq in Y for quant in seq])
    char2numY = dict(zip(v_characters, range(1,len(v_characters)+1))) # Using 0 causes trouble for tf.edit_distance
    
    # 2. Padding
    # Pad inputs
    char2numX['<GO>'] = len(char2numX) + 1
    char2numX['<PAD>'] = len(char2numX) + 1
    mx_l_X = max([len(word) for word in X]) # longest input sequence
    # Padd all X for the final form for the LSTM
    x = [[char2numX['<PAD>']]*(mx_l_X - len(word)) +[char2numX[char] for char in word] for word in X]
    x = np.array(x) 

    # Pad targets
    char2numY['<GO>'] = len(char2numY) + 1 # Define number denoting the response onset
    char2numY['<PAD>'] = len(char2numY) + 1 
    mx_l_Y = max([len(phon_seq) for phon_seq in Y]) # longest output sequence

    y = [[char2numY['<GO>']] + [char2numY['<PAD>']]*(mx_l_Y - len(ph_sq)) + [char2numY[phon] for phon in ph_sq] for ph_sq in Y]
    y = np.array(y)

    return ((x,y) , (char2numX,char2numY))




In [189]:
data = np.load('data/celex.npz')
phon_dict = np_dict_to_dict(data['phon_dict'])
word_dict = np_dict_to_dict(data['word_dict'])


datao = np.load('data/celex_old.npz')
phon_dicto = np_dict_to_dict(datao['phon_dict'])
word_dicto = np_dict_to_dict(datao['word_dict'])

In [202]:
path = "/Users/jannisborn/Desktop/LDS_Data/celex2/german/gpl/gpl.cd"
w,p = extract_celex(path)
((x,y) , (char2numX,char2numY)) = str_to_num_dataset(w,p)
print(w[2000:2100])

['anwerfen', 'anwert', 'anwesen', 'anwesend', 'anwidern', 'anwinkeln', 'anwohnen', 'anwohner', 'anwuchs', 'anwurf', 'anwurzeln', 'anzahlen', 'anzaehlen', 'anzahl', 'anzahlung', 'anzapfen', 'anzapfung', 'anzeichen', 'anzeichnen', 'anzeigen', 'anzeige', 'anzeigenblatt', 'anzeigenteil', 'anzeigepflicht', 'anzeiger', 'anziehen', 'anziehung', 'anzielen', 'anzischen', 'anzucht', 'anzuchtgarten', 'anzug', 'anzueglich', 'anzueglichkeit', 'anzuenden', 'anzuender', 'anzwecken', 'anzweifeln', 'anzwitschern', 'aeon', 'aeonenlang', 'aorist', 'aorta', 'aortenklappe', 'apathie', 'apathisch', 'apern', 'apfel', 'apfelbaum', 'aepfelchen', 'apfelfoermig', 'apfelgelee', 'apfelmost', 'apfelmus', 'apfelschimmel', 'apfelsine', 'apfelstrudel', 'apfelwein', 'aphthe', 'aphthenseuche', 'apparat', 'apparativ', 'apparatur', 'appell', 'appellation', 'appellativ', 'appellieren', 'appendix', 'appendizitis', 'applaus', 'apport', 'apportieren', 'april', 'aprilscherz', 'apriltag', 'aprilwetter', 'aquatisch', 'ar', 'ara'